In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nba-datasets/Jogos.parquet
/kaggle/input/nba-datasets/BoxScores.parquet


In [2]:
!pip install nba_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 312.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for nba_api: filename=nba_api-1.1.11-py3-none-any.whl size=251504 sha256=51ea525659e5e3f3956ece9c233705c11f46220cc14a23fab37fede41a58e7f7
  Stored in directory: /root/.cache/pip/wheels/e3/f4/46/996128675381abc175e306642bae575d372dcf7efc4683e551
Successfully built nba_api


In [3]:
import pandas as pd, time
from nba_api.stats.endpoints import leaguegamelog, boxscoretraditionalv2

pd.set_option('display.max_columns', None)

games = pd.read_parquet('/kaggle/input/nba-datasets/Jogos.parquet')
games_last_season = games['SEASON_ID'].astype(str).str[1:].astype(int).max()
boxscores = pd.read_parquet('/kaggle/input/nba-datasets/BoxScores.parquet')

games = games[games['SEASON_ID'].astype(str).str[1:].astype(int) < games_last_season]

seasons = [season for season in range(games_last_season,3000)]
#seasons = [season for season in range(2020,3000)]
season_types = ['Pre Season', 'Regular Season', 'Playoffs']

for season_type in season_types:
    for season in seasons:
        games_season = leaguegamelog.LeagueGameLog(player_or_team_abbreviation='T', season=season, season_type_all_star=season_type).league_game_log.get_data_frame()
        if not games_season.empty:
            print('{0} de {1} atualizada.'.format(season_type, season))
            games = pd.concat([games, games_season])
            games = games[-games['WL'].isnull()]
            time.sleep(0.6)
        else:
            break

games['SEASON_ID'] = games['SEASON_ID'].astype(int)
games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])
games.to_parquet('/kaggle/working/Jogos.parquet', index=False)
i = 0
while True:
    boxscores_games_id = boxscores.GAME_ID
    games_to_update_id = games[-games.GAME_ID.isin(boxscores_games_id)]

    if games_to_update_id.empty:
        print('BoxScores atualizadas')
        break

    game_to_update_id = games_to_update_id.GAME_ID.min()
    
    print('Atualizando jogo {0}'.format(game_to_update_id))

    try:
        boxscore_update = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_to_update_id).player_stats.get_data_frame()
    except:
        print('Erro na API. Tentando novamente...')
        time.sleep(60)
        continue

    if boxscore_update.empty:
        boxscore_update = pd.DataFrame(data={'GAME_ID':[game_to_update_id]})
    else:
        try:
            boxscore_update['MIN'] = boxscore_update['MIN'].astype('str').str.replace('None','0').str.split(':', expand=True).pipe(lambda x: x[0].astype(float)+x[1].astype(float)/60)
        except:
            pass
    
    boxscores = pd.concat([boxscores, boxscore_update])
    i=i+1
    if i >= 100:
        boxscores.to_parquet('/kaggle/working/BoxScores.parquet', index=False)
        i = 0
        continue

boxscores.to_parquet('/kaggle/working/BoxScores.parquet', index=False)

Pre Season de 2021 atualizada.
Regular Season de 2021 atualizada.
Playoffs de 2021 atualizada.
BoxScores atualizadas
